In [1]:
!sudo -H pip install Augmentor
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/yo'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
import torch
import random
import torchvision
import pandas as pd
import torch.nn as nn
import torchvision.models as m
import torch.nn.functional as F

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

            
def initialize_model(use_pretrained=False, num_classes=2):
    model = m.resnet50(pretrained=use_pretrained)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    return model, input_size

import os
DIR = os.getcwd()
model_dir = DIR + '/gdrive/My Drive/yo/datasets/covid19_vs_normal'
#PATH_MODEL = f'{model_dir}/checkpoints/chk_resnet_50_epoch_14.pt'
PATH_MODEL = f'{model_dir}/checkpoints/check.pt'
model, input_size = initialize_model()
#model.load_state_dict(torch.load(PATH_MODEL), strict=False)

def get_prediction(model, x):
    pred_label = model(x).argmax()
    preds = F.softmax(model(torch.rand(1,3,224,224)))
    return pred_label, preds


x = torch.rand(1,3,224,224)

get_prediction(model, x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(tensor(0), tensor([[0.7410, 0.2590]], grad_fn=<SoftmaxBackward>))

In [3]:

import time
import os
import copy
import torch
import random
import torchvision
import pandas as pd
import torch.nn as nn
import seaborn as sns
from PIL import Image
from glob import glob
import torch.optim as optim
import torchvision.models as m
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from sklearn.metrics import classification_report, confusion_matrix

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
random.seed = 2020


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from Augmentor import Pipeline
DIR = os.getcwd()

p = Pipeline(source_directory=f'{DIR}/gdrive/My Drive/yo/datasets/covid19_vs_normal/xray_dataset/val',output_directory=f'{DIR}/gdrive/My Drive/yo/datasets/covid19_vs_normal/xray_dataset/test')

#Pipeline
p.rotate(probability=0.5, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.7, min_factor=1.1, max_factor=1.2)
p.flip_left_right(probability=0.2)

p.sample(200)


# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    
    model_ft = m.resnet50(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    
    return model_ft, input_size


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = m.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = m.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = m.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = m.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = m.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = m.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


models = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']
model_name = models[0]
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}



def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:

                device = torch.device("cpu")
                #inputs = inputs.to(device)
                #labels = labels.to(device)

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

import os
DIR = os.getcwd()
data_dir = DIR+'/gdrive/My Drive/yo/datasets/covid19_vs_normal/xray_dataset'
# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val', 'test']}

# Detect if we have a GPU available
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


PyTorch Version:  1.4.0
Torchvision Version:  0.5.0


Executing Pipeline:   0%|          | 0/200 [00:00<?, ? Samples/s]

Initialised with 22 image(s) found.
Output directory set to /content/gdrive/My Drive/yo/datasets/covid19_vs_normal/xray_dataset/test.

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=L size=1802x1422 at 0x7F2C4ED60898>: 100%|██████████| 200/200 [00:40<00:00,  4.89 Samples/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [4]:
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.6818 Acc: 0.6000
val Loss: 0.8458 Acc: 0.5000

Epoch 1/14
----------
train Loss: 0.6631 Acc: 0.5750
val Loss: 0.7841 Acc: 0.5000

Epoch 2/14
----------
train Loss: 0.7051 Acc: 0.5500
val Loss: 0.5646 Acc: 0.8182

Epoch 3/14
----------
train Loss: 0.5004 Acc: 0.8500
val Loss: 0.6626 Acc: 0.5000

Epoch 4/14
----------
train Loss: 0.4885 Acc: 0.8250
val Loss: 0.4359 Acc: 0.9091

Epoch 5/14
----------
train Loss: 0.3957 Acc: 0.8750
val Loss: 0.4041 Acc: 0.9091

Epoch 6/14
----------
train Loss: 0.3564 Acc: 0.9250
val Loss: 0.4549 Acc: 0.6364

Epoch 7/14
----------
train Loss: 0.3243 Acc: 0.9500
val Loss: 0.2959 Acc: 0.9545

Epoch 8/14
----------
train Loss: 0.3104 Acc: 0.8750
val Loss: 0.2528 Acc: 0.9545

Epoch 9/14
----------
train Loss: 0.2193 Acc: 0.9750
val Loss: 0.2234 Acc: 1.0000

Epoch 10/14
----------
train Loss: 0.3700 Acc: 0.8500
val Loss: 0.1910 Acc: 1.0000

Epoch 11/14
----------
train Loss: 0.2725 Acc: 0.9250
val Loss: 0.2027 Acc: 0.9545

Ep

In [0]:
import os
DIR = os.getcwd()
model_dir = DIR + '/gdrive/My Drive/yo/datasets/covid19_vs_normal'
#torch.save(model_ft.state_dict(), f'{model_dir}/checkpoints/chk_resnet_50_epoch_{14}.pt')
torch.save(model_ft.state_dict(), f'{model_dir}/checkpoints/check.pt')
input_tensor_save = torch.rand(1,3,224,224,dtype=torch.float)
script_model = torch.jit.trace(model_ft.cpu(),input_tensor_save)
script_model.save(f'{model_dir}/checkpoints/cornet.pt')

In [0]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

images, labels = zip(*dataloaders_dict['test'].dataset.imgs)
shape = (len(images), 3, 224,224)

resize = data_transforms['test']
x_test = torch.zeros(shape)
target = torch.Tensor(labels).long()
for idx, image in enumerate(images):
    x_test[idx,:,:,:] = resize(Image.open(image).convert('RGB'))
    
nb_samples = 10
nb_classes = 2
#output = model_ft(x_test.cuda())
output = model_ft(x_test)
pred = torch.argmax(output, 1)

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(target, pred.cpu())
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))


# make predictions on the testing set
class_names = ['covid','normal']
print("[INFO] evaluating network...")
print(classification_report(target, pred.cpu(),target_names=class_names))

print_confusion_matrix(cm, class_names, figsize = (10,7), fontsize=14)

In [0]:
from google.colab import files
import os
DIR = os.getcwd()
model_dir = DIR + '/gdrive/My Drive/yo/datasets/covid19_vs_normal'
PATH_MODEL = f'{model_dir}/checkpoints/cornet.pt'
files.download(PATH_MODEL)

In [0]:
from google.colab import files
import os
DIR = os.getcwd()
model_dir = DIR + '/gdrive/My Drive/yo/datasets/covid19_vs_normal'
PATH_MODEL = f'{model_dir}/checkpoints/cornet.pt'
files.download(PATH_MODEL)